In [1]:

RENDER_DB_HOST = "dpg-d4mbue3uibrs738g22qg-a.oregon-postgres.render.com"
RENDER_DB_PORT = "5432"
RENDER_DB_NAME = "normalized_db"
RENDER_DB_USER = "normalized_db_user"
RENDER_DB_PASS = "zeG2hoet9EG6pkSbTqpgOMXEu1dMnBEd" 

print("Credentials set locally for the notebook session.")

Credentials set locally for the notebook session.


In [2]:
import pandas as pd
from sujal_codio_project import create_connection, ex1, ex2, ex3
import psycopg2
from psycopg2 import Error

def create_pg_connection():
    """Establishes a connection to the Render PostgreSQL database."""
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=RENDER_DB_NAME,
            user=RENDER_DB_USER,
            password=RENDER_DB_PASS,
            host=RENDER_DB_HOST,
            port=RENDER_DB_PORT
        )

        print("Connection to Render successful.")
        return conn

    except Error as e:
        print(f"PostgreSQL Connection Error: {e}")
        return None


def create_table(conn, create_table_sql, drop_table_name=None):
    """Creates a table, optionally dropping the old one."""
    if conn is None:
        print("Connection is None, skipping table creation.")
        return

    try:
        cursor = conn.cursor()
        if drop_table_name:
            
            cursor.execute(f'DROP TABLE IF EXISTS "{drop_table_name}" CASCADE;')
            print(f"Dropped table: {drop_table_name}")
        
        cursor.execute(create_table_sql)
        conn.commit() 
        print(f"Table created/updated.")
        
    except Error as e:
        print(f"Error executing SQL for table creation: {e}")
        conn.rollback()
    finally:
        cursor.close()

In [3]:
conn = create_pg_connection()
conn


Connection to Render successful.


<connection object at 0x0000024259A32AC0; dsn: 'user=normalized_db_user password=xxx dbname=normalized_db host=dpg-d4mbue3uibrs738g22qg-a.oregon-postgres.render.com port=5432', closed: 0>

In [4]:
cursor = conn.cursor()
cursor.execute("SELECT NOW();")
result = cursor.fetchall()
cursor.close()

result

[(datetime.datetime(2025, 11, 30, 23, 48, 56, 893462, tzinfo=datetime.timezone.utc),)]

In [21]:
import pandas as pd
from sujal_codio_project import create_connection, ex1, ex2, ex3

DB_PATH = r"C:\Users\sujal\EAS503-project-main\normalized.db"

conn = create_connection(DB_PATH)
conn

In [22]:
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table' ORDER BY NAME",
    conn
)
tables
print(tables.head())

              name
0          Country
1         Customer
2      OrderDetail
3          Product
4  ProductCategory


In [23]:
sql = ex3(conn)
print(sql)

df = pd.read_sql_query(sql, conn)
df.head()



    SELECT 
        Cust.FirstName || ' ' || Cust.LastName as Name,
        ROUND(SUM(Prod.ProductUnitPrice * Ord.QuantityOrdered), 2) as Total
    FROM OrderDetail Ord
    JOIN Customer Cust ON Ord.CustomerID = Cust.CustomerID
    JOIN Product Prod ON Ord.ProductID = Prod.ProductID
    GROUP BY Name
    ORDER BY Total DESC
    


,Name,Total
0,Annette Roulet,6092806.79
1,Manuel Pereira,5905784.45
2,Janete Limeira,5817785.32
3,Christina Berglund,5679042.68
4,Liz Nixon,5623333.51


In [13]:
customers = pd.read_sql_query(
    "SELECT FirstName || ' ' || LastName AS Name FROM Customer ORDER BY Name LIMIT 20;",
    conn
)
customers.head()

DatabaseError: Execution failed on sql 'SELECT FirstName || ' ' || LastName AS Name FROM Customer ORDER BY Name LIMIT 20;': no such table: Customer